# Zagadki

![image-2.png](attachment:image-2.png)

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

# Kod startowy

In [185]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [186]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm

## Ładowanie danych

In [187]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
path_to_data = 'data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

In [188]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)

    if ret:
        ret = ret.split("_")
        return ret[0]
        # return ret
    return word

In [189]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

Retrieving folder contents


Processing file 1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI plwiktionary_definitions_clean.txt
Processing file 1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk superbazy_clean.txt
Processing file 1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh w2v_polish_lemmas.model
Processing file 1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC w2v_polish_lemmas.model.syn1neg.npy
Processing file 1RY0Ftfx_-nPUbddYCvHq9p8yCcXbUrYS w2v_polish_lemmas.model.wv.vectors.npy
Processing file 18wrF9VyESTvIT9Z_TqUd_2wV4lP2As3t zagadki_do_testow_clean.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI
To: /content/data/zagadki/plwiktionary_definitions_clean.txt
100%|██████████| 1.59M/1.59M [00:00<00:00, 88.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk
To: /content/data/zagadki/superbazy_clean.txt
100%|██████████| 43.7M/43.7M [00:00<00:00, 72.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh
To: /content/data/zagadki/w2v_polish_lemmas.model
100%|██████████| 40.8M/40.8M [00:00<00:00, 53.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC
From (redirected): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC&confirm=t&uuid=b9306bf3-f211-445d-95b1-f65387e343cc
To: /content/data/zagadki/w2v_polish_lemmas.model.syn1neg.npy
100%|██████████| 958M/958M

In [190]:
files = os.listdir(path_to_data + "zagadki")
print("Pobrane pliki:")
for file in files:
    print(file)



Pobrane pliki:
superbazy_clean.txt
zagadki_do_testow_clean.txt
w2v_polish_lemmas.model
plwiktionary_definitions_clean.txt
w2v_polish_lemmas.model.syn1neg.npy
w2v_polish_lemmas.model.wv.vectors.npy


In [191]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

In [192]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
model = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

In [205]:
categories = {
    "{{techn}}": "technika, techniczny",
    "{{B}}": "biernik",
    "{{zootechn}}": "zootechnika, zootechniczny",
    "{{stpol}}": "staropolski",
    "{{więz}}": "więziennictwo, więzienny",
    "{{kartogr}}": "kartografia, kartograficzny",
    "{{przym}}": "przymiotnik",
    "{{ekon}}": "ekonomia, ekonomiczny",
    "{{mitgerm}}": "mitologia germańska",
    "{{amer}}": "amerykański angielski",
    "{{sf}}": "science fiction",
    "{{kujawy}}": "gwara kujawska",
    "{{ż}}": "rodzaj żeński",
    "{{przestęp}}": "przestępczość, przestępczy",
    "{{miner}}": "mineralogia, mineralogiczny",
    "{{księg}}": "księgowość, księgowy",
    "{{nłac}}": "nowoczesna łacina",
    "{{psych}}": "psychologia, psychologiczny",
    "{{nawig}}": "nawigacja, nawigacyjny",
    "{{fonet}}": "fonetyka, fonetyczny",
    "{{dypl}}": "dyplomacja",
    "{{fakt}}": "faktyczny",
    "{{zdrobn}}": "zdrobnienie",
    "{{hotel}}": "hotelarstwo, hotelarski",
    "{{praw}}": "prawo, prawniczy",
    "{{mitgr}}": "mitologia grecka",
    "{{plast}}": "plastyka, plastyczny",
    "{{syst}}": "systematyka",
    "{{monet}}": "numizmatyka, numizmatyczny",
    "{{muz}}": "muzyka, muzyczny",
    "{{socjol}}": "socjologia, socjologiczny",
    "{{itp}}": "i tym podobne",
    "{{zool}}": "zoologia, zoologiczny",
    "{{zgrub}}": "zgrubienie",
    "{{paleont}}": "paleontologia, paleontologiczny",
    "{{pot}}": "potocznie",
    "{{mit}}": "mitologia",
    "{{speleol}}": "speleologia, speleologiczny",
    "{{chem}}": "chemia, chemiczny",
    "{{grub}}": "grubiański",
    "{{n}}": "rodzaj nijaki",
    "{{kult}}": "kultura, kulturoznawstwo, kulturoznawczy",
    "{{metrol}}": "metrologia, metrologiczny",
    "{{szkocang}}": "szkocki angielski",
    "{{astronaut}}": "astronautyka, astronautyczny",
    "{{bot}}": "botanika, botaniczny",
    "{{łow}}": "łowiectwo, łowiecki",
    "{{urb}}": "urbanistyka, urbanistyczny",
    "{{filoz}}": "filozofia, filozoficzny",
    "{{tosk}}": "toskański",
    "{{zarz}}": "zarządzanie, zarządczy",
    "{{płn}}": "północny",
    "{{ezot}}": "ezoteryka",
    "{{mitrzym}}": "mitologia rzymska",
    "{{poet}}": "poetycki",
    "{{dendr}}": "dendrologia, drzewoznawstwo, dendrologiczny, drzewoznawczy",
    "{{odczas}}": "odczasownikowy",
    "{{iron}}": "ironiczny, ironicznie",
    "{{reg}}": "regionalizm",
    "{{mar}}": "marynistyka, marynistyczny",
    "{{kulin}}": "kulinaria, kulinarny",
    "{{mitsłow}}": "mitologia słowiańska",
    "{{fot}}": "fotografia",
    "{{żegl}}": "żegluga, żeglarski",
    "{{miar}}": "miernictwo",
    "{{fż}}": "fizyka",
    "{{żyd}}": "żydowski",
    "{{hutn}}": "hutnictwo, hutniczy",
    "{{przesz}}": "przestarzały",
    "{{kib}}": "kibicostwo, kibicowski",
    "{{myśl}}": "myślistwo, łowiectwo",
    "{{pejor}}": "pejoratywny, pejoratywnie",
    "{{ogrod}}": "ogrodnictwo, ogrodniczy",
    "{{itd}}": "i tak dalej",
    "{{dial}}": "dialekt",
    "{{jeźdz}}": "jeździectwo, jeździecki",
    "{{flis}}": "flisactwo",
    "{{zobacz}}": "zobacz",
    "{{gwara}}": "gwarowe",
    "{{rel}}": "religia, religijny",
    "{{herp}}": "herpetologia, herpetologiczny",
    "{{adm}}": "administracja (państwowa, rządowa itd.)",
    "{{icht}}": "ichtiologia, ichtiologiczny",
    "{{także}}": "także",
    "{{telew}}": "telewizja, telewizyjny",
    "{{m}}": "rodzaj męski",
    "{{gin}}": "ginekologia, ginekologiczny",
    "{{pogard}}": "pogardliwy, pogardliwie",
    "{{odl}}": "odlewnictwo, odlewniczy",
    "{{jubil}}": "jubilerstwo, jubilerski",
    "{{biur}}": "biurowość, biurowy",
    "{{ryb}}": "rybołówstwo, rybołówczy",
    "{{obraź}}": "obraźliwy, obraźliwie",
    "{{rzem}}": "rzemiosło, rzemieślniczy",
    "{{hig}}": "higiena, higieniczny",
    "{{spoż}}": "spożywczy",
    "{{harc}}": "harcerstwo, harcerski",
    "{{peryfr}}": "peryfraza",
    "{{fryzj}}": "fryzjerstwo, fryzjerski",
    "{{środ}}": "środowiskowy",
    "{{bryt}}": "brytyjski angielski",
    "{{liter}}": "literacki",
    "{{jęz}}": "językoznawstwo, językoznawczy",
    "{{krym}}": "kryminalistyka, kryminologiczny",
    "{{ts}}": "taoizm",
    "{{poznań}}": "regionalizm poznański",
    "{{szach}}": "szachy, szachowy",
    "{{wikipedia}}": "Wikipedia",
    "{{herald}}": "heraldyka, heraldyczny",
    "{{kynol}}": "kynologia, kynologiczny",
    "{{gastr}}": "gastronomia, gastronomiczny",
    "{{bud}}": "budownictwo",
    "{{zaw}}": "zawodowy",
    "{{log}}": "logika, logiczny",
    "{{stud}}": "studencki",
    "{{kraków}}": "regionalizm krakowski",
    "{{fant}}": "fantastyka",
    "{{austral}}": "australijski; język angielski używany w Australii",
    "{{ręk}}": "rękodzieło, rękodzielniczy",
    "{{żart}}": "żartobliwy, żartobliwie",
    "{{infant}}": "infantylny",
    "{{meteorol}}": "meteorologia, meteorologiczny",
    "{{typogr}}": "typografia, typograficzny",
    "{{kośc}}": "kościelny",
    "{{uczn}}": "uczniowski",
    "{{turyst}}": "turystyka, turystyczny",
    "{{geol}}": "geologia, geologiczny",
    "{{publ}}": "publicystyka, publicystyczny",
    "{{antrop}}": "antropologia, antropologiczny",
    "{{łódź}}": "regionalizm łódzki",
    "{{kraw}}": "krawiectwo, krawiecki",
    "{{hist}}": "historia, historyczny",
    "{{symbol}}": "symboliczny",
    "{{geod}}": "geodezja, geodezyjny",
    "{{stomat}}": "stomatologia, stomatologiczny",
    "{{geofiz}}": "geofizyka, geofizyczny",
    "{{starop}}": "staropolski",
    "{{biol}}": "biologia, biologiczny",
    "{{polit}}": "polityka, polityczny",
    "{{choreogr}}": "choreografia, choreograficzny",
    "{{telekom}}": "telekomunikacja, telekomunikacyjny",
    "{{agrot}}": "agrotechnika, agrotechniczny",
    "{{anat}}": "anatomia, anatomiczny",
    "{{geogr}}": "geografia, geograficzny",
    "{{os}}": "osobowy",
    "{{mitbask}}": "mitologia baskijska",
    "{{wędk}}": "wędkarstwo, wędkarski",
    "{{roln}}": "rolnictwo, rolniczy",
    "{{biochem}}": "biochemia, biochemiczny",
    "{{opt}}": "optyka, optyczny",
    "{{fryz}}": "język fryzyjski",
    "{{pedag}}": "pedagogika, pedagogiczny",
    "{{przest}}": "przestarzały",
    "{{bibliot}}": "bibliotekoznawstwo, bibliotekoznawczy",
    "{{mors}}": "język morski",
    "{{daw}}": "dawniej, dawny",
    "{{eduk}}": "edukacja, edukacyjny",
    "{{slang}}": "slang, slangowy",
    "{{warmia}}": "gwara warmińska",
    "{{wet}}": "weterynaria, weterynaryjny",
    "{{kolej}}": "kolejnictwo, kolejowy",
    "{{lit}}": "język litewski",
    "{{kosmet}}": "kosmetyka, kosmetyczny",
    "{{poligr}}": "poligrafia, poligraficzny",
    "{{elektr}}": "elektryczność, elektryczny",
    "{{hand}}": "handel, handlowy",
    "{{wulg}}": "wulgarny",
    "{{mech}}": "mechanika, mechaniczny",
    "{{etn}}": "etnografia, etnograficzny",
    "{{społ}}": "społeczny",
    "{{górn}}": "górnictwo, górniczy",
    "{{rub}}": "rubaszny",
    "{{eufem}}": "eufemistycznie",
    "{{etc}}": "et cetera",
    "{{bibl}}": "biblijny",
    "{{ekol}}": "ekologia, ekologiczny",
    "{{D}}": "dopełniacz",
    "{{książk}}": "książkowy",
    "{{wojsk}}": "wojskowość, wojskowy",
    "{{lotn}}": "lotnictwo, lotniczy",
    "{{neol}}": "neologizm",
    "{{szt}}": "sztuka",
    "{{urz}}": "urzędniczy, urzędowy",
    "{{bizk}}": "dialekt biskajski języka baskijskiego",
    "{{poczt}}": "poczta, pocztowy",
    "{{W}}": "wołacz",
    "{{kanadang}}": "kanadyjski angielski",
    "{{bank}}": "bankowość, bankowy",
    "{{staroż}}": "starożytny",
    "{{wiośl}}": "wioślarstwo, wioślarski",
    "{{karc}}": "karczmarstwo, karczma",
    "{{archit}}": "architektura, architektoniczny",
    "{{med}}": "medycyna, medyczny",
    "{{dosł}}": "dosłownie",
    "{{kresy}}": "kresowy",
    "{{akust}}": "akustyka, akustyczny",
    "{{leśn}}": "leśnictwo, leśniczy",
    "{{przen}}": "przenośny, przenośnie",
    "{{mikol}}": "mikologia, mikologiczny",
    "{{współcz}}": "współczesny",
    "{{posp}}": "pospolity",
    "{{fiz}}": "fizyka, fizyczny",
    "{{nauk}}": "naukowy",
    "{{ter}}": "terminologia",
    "{{mot}}": "motoryzacja, motoryzacyjny",
    "{{astr}}": "astronomia, astronomiczny",
    "{{film}}": "filmologia, filmoznawstwo, filmologiczny, filmoznawczy, filmowy",
    "{{teol}}": "teologia, teologiczny",
    "{{lud}}": "ludowy, ludoznawstwo, ludoznawczy",
    "{{mat}}": "matematyka, matematyczny",
    "{{ofic}}": "oficjalny",
    "{{mikrobiol}}": "mikrobiologia, mikrobiologiczny",
    "{{archeol}}": "archeologia, archeologiczny",
    "{{Gloger}}": "Gloger",
    "{{mebl}}": "meblarstwo, meblarski",
    "{{młodz}}": "młodzieżowy",
    "{{brazport}}": "język portugalski, używany w Brazylii",
    "{{M}}": "mianownik",
    "{{pszcz}}": "pszczelarstwo, pszczelarski",
    "{{teatr}}": "teatr, teatralny",
    "{{pieszcz}}": "pieszczotliwy",
    "{{lp}}": "liczba pojedyncza",
    "{{podn}}": "podniosły, górnolotny",
    "{{met}}": "metalurgia, metalurgiczny",
    "{{gram}}": "gramatyka, gramatyczny",
    "{{lwów}}": "regionalizm lwowski",
    "{{stat}}": "statystyka, statystyczny",
    "{{sport}}": "sport, sportowy",
    "{{hydrol}}": "hydrologia, hydrologiczny",
    "{{geom}}": "geometria",
    "{{śrłac}}": "średniowieczna łacina",
    "{{białystok}}": "regionalizm białostocki",
    "{{grzecz}}": "zwrot grzecznościowy",
    "{{ekspr}}": "ekspresywnie, ekspresywny",
    "{{seks}}": "seksualny",
    "{{farm}}": "farmacja, lekoznawstwo, farmakologia, farmaceutyczny, farmakologiczny",
    "{{rzad}}": "rzadki",
    "{{druk}}": "drukarstwo",
    "{{fizj}}": "fizjologia, fizjologiczny",
    "{{np}}": "liczba mnoga",
    "{{zwł}}": "zwłaszcza",
    "{{filatel}}": "filatelistyka",
    "{{blp}}": "bez liczby pojedynczej",
    "{{folk}}": "folklor",
    "{{włók}}": "włókiennictwo, włókienniczy",
    "{{przysz}}": "przyszły",
    "{{elektron}}": "elektronika, elektroniczny",
    "{{masz}}": "maszynoznawstwo, maszynowy",
    "{{odprzym}}": "odczasownikowy przymiotnik",
    "{{demogr}}": "demografia",
    "{{enol}}": "enologia, winoznawstwo, enologiczny, winoznawczy",
    "{{lm}}": "liczba mnoga",
    "{{ent}}": "entomologia, owadoznawstwo, entomologiczny, owadoznawczy",
    "{{finans}}": "finanse, finansowy",
    "{{rad}}": "radiotechnika, radiotechniczny",
    "{{inform}}": "informatyka, informatyczny",
    "{{rzecz}}": "rzeczownik",
    "{{gip}}": "dialekt gipuzkoański języka baskijskiego",
    "{{właśc}}": "właściwy",
    "{{spec}}": "specjalny",
    "{{numizm}}": "numizmatyka, numizmatyczny",
    "{{ornit}}": "ornitologia, ornitologiczny",
    "{{cuk}}": "cukiernictwo",
    "{{lekcew}}": "lekceważący"
}

In [194]:
import re

######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')

    definition = definition.replace("'", " ")
    for cat in re.findall(r"{{\w+}}", definition):
        definition = definition.replace(cat, categories[cat])

    L = word.split()
    if len(L) == 1:
        word = L[0]

        definition = set(tokenize(definition.lower()))
        all_word_definitions[word].append(definition)
        for word in set(definition):
            base_idf[get_word_base(word)] += 1


for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

In [195]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()
        answers.append(line[0])
        queries.append(tokenize(' '.join(line[1:])))

## Kod z kryteriami oceniającymi

In [196]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)

    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)

    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [197]:
def clear_riddle(riddle):
    cleared_riddle = np.array([])

    # for word in riddle:
    #     cleared_riddle = np.append(cleared_riddle, word.replace("'", " "). replace('"'," "))

    # list_to_remove = [",",".","?","-", "czy", "i", "{", "}"]
    list_to_remove = [",",".","?", "!", "#", "%", "'", "''", '"', '""',"-", "czy", "i", "{", "}", ";", "w", "lub", "(", ")", "wikipedia", "z", "do", "na", ".", ":", "_", "-","/","@", "^", "&", "*","+"]
    for el in list_to_remove:
        riddle.discard(el)

    for word in riddle:
      cleared_riddle = np.append(cleared_riddle, get_word_base(word))

    return cleared_riddle

def vectorize_sentence(riddle):
    global model

    words = np.array([])
    weights = np.array([])

    for word in riddle:
        if (word in model.wv):
            words = np.append(words, word)
            if(word in base_idf):
                if(base_idf[word]<0):
                    weights = np.append(weights, 0)
                else:
                    weights = np.append(weights, base_idf[word])
            else:
                weights = np.append(weights, 34)
    if (len(words)>0):
        # print(weights)
        # print(words)

        word_vectors = np.array([model.wv[word] for word in words])

        average_vector = np.average(word_vectors, axis=0, weights=weights)

        return average_vector
    return np.nan

def closest_words_to_vectorized_string(riddle,vectorized_string, topn=20):
    global model

    closest_words = []
    for word, definitions in all_word_definitions.items():
        if(get_word_base(word) in model.wv and get_word_base(word) not in riddle):
            word_vector = model.wv[get_word_base(word)]
            similarity = np.dot(vectorized_string, word_vector) / (np.linalg.norm(vectorized_string) * np.linalg.norm(word_vector))
            closest_words.append((word, similarity))
            if(word in vectorized_definitions):
                for word_vector2 in vectorized_definitions[word]:
                    similarity = np.dot(vectorized_string, word_vector2) / (np.linalg.norm(vectorized_string) * np.linalg.norm(word_vector2))
                    closest_words.append((word, similarity))

    closest_words.sort(key=lambda x: x[1], reverse=True)

    closest = {}

    for (word, value) in closest_words:
        if word not in closest.keys():
            closest[word] = value
        if len(closest) > 19:
            break

    closest_words = [(key, value) for key, value in closest.items()]


    closest_words.sort(key=lambda x: x[1], reverse=True)



    return closest_words

In [198]:
answer_riddle(set(tokenize("wielostronicowa duża księga, tomisko")),20)

['bok',
 'herbarz',
 'ekstrakt',
 'dekret',
 'dziennik',
 'kronika',
 'agenda',
 'kanon',
 'konto',
 'książeczka',
 'notatnik',
 'wpis',
 'papirus',
 'blok',
 'formularz',
 'wyciąg',
 'druk',
 'suplement',
 'katalog',
 'manuskrypt']

In [199]:
# base_idf2 = base_idf.copy();
# list(filter(lambda x: len(x[0])<2,sorted(base_idf2.items(), key = lambda x: x[1])))

In [200]:
import numpy as np
vectorized_definitions = {}

for word, definitions in all_word_definitions.items():
            for single_definition in definitions:
                if (len(single_definition) != 0):
                    word_vector = vectorize_sentence(clear_riddle(single_definition))
                    if (word_vector is not np.nan):
                        if (word in vectorized_definitions):
                            vectorized_definitions[word] = np.append(vectorized_definitions[word],[word_vector], axis=0)
                        else:
                            vectorized_definitions[word] = np.array([word_vector])

In [201]:
# ///////////////////////////////////////\
#
# PAMIĘTAJ O ZMIANIE W get_word_base()
# I o zmianie w definiton.replace("'"," ")
#
# //////////////////////////////////////

import numpy as np

def get_word_base_my(word):
    return get_word_base(word)

def answer_riddle(riddle, K):
    global model

    riddle = clear_riddle(riddle)


    average_vector = vectorize_sentence(riddle)

    similar_words_with_probabilities = closest_words_to_vectorized_string(riddle,average_vector, K)

    # similar_words_with_probabilities = model.wv.similar_by_vector(average_vector, topn=K)
    similar_words = [word[0] for word in similar_words_with_probabilities]

    return similar_words

# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [202]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    i=0
    for query in tqdm(queries, desc="queries answered"):
        computed_answers.append(score_function(set(query), K=K))
        # print(queries[i])
        # print(answers[i])
        # print(computed_answers[i])
        # print("///////////////////////////////////")
        i+=1
    score = mean_reciprocal_rank(answers, computed_answers, K=K)

    return score

In [204]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    PART_OF_DATA = 100
    START = 0
    K = 20
    valid_queries = queries[START:PART_OF_DATA]
    valid_answers = answers[START:PART_OF_DATA]

    # for i in range(PART_OF_DATA):
    #   print(valid_queries[i])
    #   print(valid_answers[i])
    #   print(all_word_definitions[valid_answers[i]])

    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered: 100%|██████████| 100/100 [00:59<00:00,  1.68it/s]

Mean Reciprocal Rank = 0.22959060238007598
Score: 0.22959060238007598


# ZRÓB KOPIĘ

Zrób kopięęęęęęęęęęęęęęęęę